# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier



---
## Step 0: Load The Data

In [ ]:
# Load pickled data
import pickle

# TODO: Fill this in based on where you saved the training and testing data

training_file = "data/train.p"
validation_file= "data/valid.p"
testing_file = "data/test.p"

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

---

## Step 1: Dataset Summary & Exploration


### Basic Summary of the Data Set Using Python and Numpy 

In [ ]:
### Replace each question mark with the appropriate value. 
### Use python, pandas or numpy methods rather than hard coding the results

# TODO: Number of training examples
n_train = len(X_train)

# TODO: Number of validation examples
n_validation = len(X_valid)

# TODO: Number of testing examples.
n_test = len(X_test)

# TODO: What's the shape of an traffic sign image?
image_shape = X_test.shape

# TODO: How many unique classes/labels there are in the dataset.
n_classes = len(set(y_train))

print("Number of training examples =", n_train)
print("Number of validation examples =", n_validation)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

### Include an exploratory visualization of the dataset

In [ ]:
### Data exploration visualization code goes here.
### Feel free to use as many code cells as needed.
import matplotlib.pyplot as plt
import random
from skimage.transform import rotate

# Visualizations will be shown in the notebook.
%matplotlib inline


plt.hist(y_test ,bins=range(0,n_classes),normed=True,alpha=0.25)
plt.title("Test set distribution of classes")
plt.show()
plt.hist(y_valid,bins=range(0,n_classes),normed=True,alpha=0.25)
plt.title("Validation set distribution of classes")
plt.show()
plt.hist(y_train,bins=range(0,n_classes),normed=True,alpha=0.25)
plt.title("Training set distribution of classes")
plt.show()
plt.title("Random sign from training set")
random_image_i = random.randrange(n_train)
plt.imshow(X_train[random_image_i])
print("image {}".format(random_image_i))


----

## Step 2: Design and Test a Model Architecture


### Pre-process the Data Set (normalization, grayscale, etc.)

In [ ]:
### Preprocess the data here. It is required to normalize the data. Other preprocessing steps could include 
### converting to grayscale, etc.
### Feel free to use as many code cells as needed.
from sklearn.utils import shuffle
import cv2
import numpy as np

X_train, y_train = shuffle(X_train, y_train)

def normalize(data):
    return (data - 128.0) / 128.0

def toGray(data):
    x = []
    
    for i in range(0,len(data)):
        x.append(cv2.cvtColor(data[i], cv2.COLOR_RGB2GRAY).reshape((32,32,1)))
                 
    return np.array(x)

def preprocess(images):
    return normalize(toGray(images))

X_train = preprocess(X_train)
X_valid = preprocess(X_valid)
X_test = preprocess(X_test)




### Model Architecture

In [ ]:
import tensorflow as tf

EPOCHS = 50
BATCH_SIZE = 128


from tensorflow.contrib.layers import flatten

def MccNet(x,dropout_prob):    
    # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
    mu = 0
    sigma = 0.1
    
    # SOLUTION: Layer 1: Convolutional. Input = 32x32x1. Output = 28x28x6.
    conv1_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 1, 6), mean = mu, stddev = sigma))
    conv1_b = tf.Variable(tf.zeros(6))
    conv1   = tf.nn.conv2d(x, conv1_W, strides=[1, 1, 1, 1], padding='VALID') + conv1_b

    # SOLUTION: Activation.
    conv1 = tf.nn.relu(conv1)

    # SOLUTION: Pooling. Input = 28x28x6. Output = 14x14x6.
    conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    #conv1 = tf.layers.dropout(conv1, dropout_prob)
    # SOLUTION: Layer 2: Convolutional. Output = 10x10x16.
    conv2_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 6, 16), mean = mu, stddev = sigma))
    conv2_b = tf.Variable(tf.zeros(16))
    conv2   = tf.nn.conv2d(conv1, conv2_W, strides=[1, 1, 1, 1], padding='VALID') + conv2_b
    
    # SOLUTION: Activation.
    conv2 = tf.nn.relu(conv2)

    # SOLUTION: Pooling. Input = 10x10x16. Output = 5x5x16.
    conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    #conv2 = tf.layers.dropout(conv2, dropout_prob)
    
    # SOLUTION: Flatten. Input = 5x5x16. Output = 400.
    fc0   = flatten(conv2)
    
    # SOLUTION: Layer 3: Fully Connected. Input = 400. Output = 120.
    fc1_W = tf.Variable(tf.truncated_normal(shape=(400, 120), mean = mu, stddev = sigma))
    fc1_b = tf.Variable(tf.zeros(120))
    fc1   = tf.matmul(fc0, fc1_W) + fc1_b
    
    # SOLUTION: Activation.
    fc1    = tf.nn.relu(fc1)
    
    fc1 = tf.layers.dropout(fc1, dropout_prob)

    # SOLUTION: Layer 4: Fully Connected. Input = 120. Output = 84.
    fc2_W  = tf.Variable(tf.truncated_normal(shape=(120, 84), mean = mu, stddev = sigma))
    fc2_b  = tf.Variable(tf.zeros(84))
    fc2    = tf.matmul(fc1, fc2_W) + fc2_b
    
    # SOLUTION: Activation.
    fc2    = tf.nn.relu(fc2)
    
    fc2 = tf.layers.dropout(fc2, dropout_prob)

    # Layer 5: Fully Connected. Input = 84. Output = 43.
    fc3_W  = tf.Variable(tf.truncated_normal(shape=(84, n_classes), mean = mu, stddev = sigma))
    fc3_b  = tf.Variable(tf.zeros(n_classes))
    logits = tf.matmul(fc2, fc3_W) + fc3_b
    
    return logits


### Train, Validate and Test the Model

In [ ]:
import tensorflow as tf
### Train your model here.
### Calculate and report the accuracy on the training and validation set.
### Once a final model architecture is selected, 
### the accuracy on the test set should be calculated and reported as well.
### Feel free to use as many code cells as needed.

x = tf.placeholder(tf.float32, shape=(None, 32, 32, 1))
y = tf.placeholder(tf.int32, shape=(None))
dropout_prob = tf.placeholder(tf.float32, shape=(None))
one_hot_y = tf.one_hot(y, n_classes)



In [ ]:
rate = 0.001

logits = MccNet(x,dropout_prob)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=one_hot_y)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

In [ ]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data, sess):
    num_examples = len(X_data)
    total_accuracy = 0
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y, dropout_prob: 1.0})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        X_train, y_train = shuffle(X_train, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y, dropout_prob: 0.5})
            
        
        training_accuracy = evaluate(X_train,y_train, sess)
        validation_accuracy = evaluate(X_valid, y_valid, sess)
        print("EPOCH {} ...".format(i+1))
        print("Training Accuracy = {:.3f}".format(training_accuracy))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print()
        
    saver.save(sess, 'mccnet')
    print("Model saved")

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    print("Training Accuracy = {:.3f}".format( evaluate(X_train, y_train, sess) ) )
    print("Validation Accuracy = {:.3f}".format( evaluate(X_valid, y_valid, sess) ) )
    print("Test Accuracy = {:.3f}".format( evaluate(X_test, y_test, sess) ) )


---

## Step 3: Test a Model on New Images


### Load and Output the Images

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.

from PIL import Image
import numpy as np

def readAll(locations, directory="./web-samples/"):
    a = []
    for location in locations:
        a.append(Image.open(directory + location))
    return a
    
def resizeAll(images):
    new_images = []
    for image in images:
        (width,height) = image.size
        
        left = (width/2)-(height/2)
        upper = 0
        right = (width/2)+(height/2)
        lower = height        
        new_image = image.crop((left,upper,right,lower))        
        new_image.thumbnail((32,32)) 
        new_images.append( new_image)
    return new_images

def show(images):
    plt.figure(1)
    for i in range(0,len(images)):
        plt.subplot(1,len(images),i + 1)
        plt.imshow(images[i] )

def toArray(images):
    a = []
    for image in images:
        image_as_array = np.array(image)
        a.append(image_as_array)
    return np.array(a)
        
def load(image_names):
    return resizeAll(readAll(image_names))



image_names = ["stop.jpg","do-not-enter.jpg","speed-limit-30.jpg","priority-road.jpg","speed-limit-60.jpg"]
images = load(image_names)
show(images)

### Predict the Sign Type for Each Image

In [ ]:
### Run the predictions here and use the model to output the prediction for each image.
### Make sure to pre-process the images with the same pre-processing pipeline used earlier.
### Feel free to use as many code cells as needed.

def load_sign_names():
    id_to_name = {}
    for line in open("./signnames.csv"):
        fields = line.split(",")
        id_to_name[fields[0]] = fields[1]
    return id_to_name



id_to_name = load_sign_names()

sample_x = toArray(images)
sample_x = preprocess(sample_x)
sample_y = np.array([14,17,1,12,3])
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    predictions = sess.run(tf.argmax(logits, 1), feed_dict={x: sample_x, dropout_prob: 1.0})
    
    print(predictions)
    for prediction in predictions:
        print("Prediction: {}".format(id_to_name[str(prediction)]))

### Analyze Performance

In [ ]:
### Calculate the accuracy for these 5 new images. 
### For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate on these new images.
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    print("Web sample accuracy: {:.2%}".format( evaluate(sample_x, sample_y, sess)) )

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

In [ ]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    softmax_probs = sess.run(tf.nn.softmax(logits), feed_dict={x: sample_x, dropout_prob:1.0})
    print(sess.run(tf.nn.top_k(tf.constant(softmax_probs), k=5)))
